In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
from itertools import product
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

# Functions

# Specifications

In [2]:
optimal_cluster = 'st_cluster_3_5_7'
name = ''
save = True

# Base CPI for adjustment (e.g., 2024)
BASE_CPI = 313.3

# Load Clustered Claims

In [3]:
claims_clusters = pd.read_csv('clustered_claims_sensitivity.csv')
# Ensure that 'countyCode' is properly formatted as a 5-character string
claims_clusters['countyCode'] = claims_clusters['countyCode'].astype(int).astype(str)
claims_clusters['countyCode'] = claims_clusters['countyCode'].apply(lambda x: str(x).zfill(5))

/tmp/ipykernel_297/2351069484.py:1: DtypeWarning: Columns (7,25,32,35,38,42,51,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_clusters = pd.read_csv('../2_Low_Return_Period/Clusters/2025_all/clustered_claims_sensitivity.csv')


# Load CPI

In [4]:
# Load CPI data
CPI_insurance = pd.read_csv("../Local_Data/BLS_Data/US_BLS_CPIAUCSL.csv", parse_dates=["DATE"]) 
#CPI_insurance = pd.read_csv("../Local_Data/BLS_Data/CPI_by_Year_Month.csv") for homeowners insurance CPI

# Ensure 'CPIAUCSL' is numeric
CPI_insurance["CPIAUCSL"] = pd.to_numeric(CPI_insurance["CPIAUCSL"], errors="coerce")

# Handle missing or non-numeric values by forward and backward filling
CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
CPI_insurance["CPIAUCSL"].fillna(method="bfill", inplace=True)

# Extract year and calculate annual average CPI
CPI_insurance['Year'] = CPI_insurance["DATE"].dt.year
CPI_insurance = CPI_insurance.groupby('Year')["CPIAUCSL"].mean().reset_index()
CPI_insurance = CPI_insurance.rename(columns={"CPIAUCSL": "CPI"})

# Filter claims_cluster to only include those from 1998 to present
claims_clusters['dateOfLoss'] = pd.to_datetime(claims_clusters['dateOfLoss'])

# Extract month (abbreviated) and year from 'dateOfLoss'
claims_clusters.loc[:, 'yearOfLoss'] = claims_clusters['dateOfLoss'].dt.year

# Merge claims_clusters with CPI_insurance on month (abbreviation) and year
claims_merged = pd.merge(claims_clusters, CPI_insurance, left_on=['yearOfLoss'], right_on=['Year'])
claims_merged = claims_merged.drop(columns=['Year'])

# Create new field for 'totalClaimPaid'
claims_merged['totalClaimPaid'] = claims_merged['amountPaidOnBuildingClaim'].fillna(0) + claims_merged['amountPaidOnContentsClaim'].fillna(0)

# Create 'adjustedClaim' column by dividing the CPI for May 2024 by the CPI at the time of loss, and multiplying by 'totalClaimPaid'
claims_merged['adjustedClaim'] = (BASE_CPI / claims_merged['CPI']) * claims_merged['totalClaimPaid']

claims_clusters = claims_merged
if save:
    claims_clusters.to_csv('clustered_claims_final.csv')

/tmp/ipykernel_297/3827790798.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_297/3827790798.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_297/3827790798.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an in